In [28]:
from dataclasses import dataclass, asdict
import requests
import time
import duckdb
import pandas as pd
from configparser import ConfigParser
import pgeocode

%load_ext sql
%config SqlMagic.displaylimit = 0

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [9]:
config = ConfigParser()
config.read("../../.config")

['../../.config']

In [10]:
config.get("GoogleMaps", "base_url")

'https://maps.googleapis.com/maps/api'

In [24]:
api_key = config['GoogleMaps']['API_KEY']
base_url = config['GoogleMaps']['base_url']
radius = 50000  # in meters
place_type = 'shopping'  # Example place type
address = 'Luxembourg City, Luxembourg'


In [12]:
def format_coordinates(latitude, longitude):
    return f"{latitude},{longitude}"

def get_coordinates(api_key, address):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
        else:
            print(f"Error in response: {data['status']}")
            return None, None
    else:
        print(f"HTTP error: {response.status_code}")
        return None, None




In [ ]:

latitude, longitude = get_coordinates(API_KEY, address)

if latitude and longitude:
    formatted_location = format_coordinates(latitude, longitude)
    # print(formatted_location)  # Output will be the coordinates of Luxembourg City
else:
    print("Could not retrieve coordinates.")

In [5]:
# url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&key={API_KEY}"
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&type={place_type}&key={API_KEY}"

response = requests.get(url)
places = response.json()

for place in places['results']:
    print(f"Name: {place['name']}")
    print(f"Address: {place.get('vicinity', 'N/A')}")
    print(f"Place ID: {place['place_id']}")
    print('---')


Name: Luxembourg
Address: Luxembourg
Place ID: ChIJVyzznc1IlUcREG0F0dbRAAQ
---
Name: Hôtel Parc Belle-Vue
Address: 5 Avenue Marie-Thérèse, Luxembourg
Place ID: ChIJ3e9sj9VIlUcRUU6bnYZ2tcE
---
Name: Hotel Vauban
Address: 10 Place Guillaume II, Luxembourg
Place ID: ChIJQVrlNdNIlUcRa3v0FrkGQ-g
---
Name: French Café Hotel
Address: 14 Place d'Armes, Luxembourg
Place ID: ChIJz1niVCtPlUcROFfnez0PXz0
---
Name: Hotel Christophe Colomb
Address: 10 Rue d'Anvers, Luxembourg
Place ID: ChIJHeKzA9FIlUcR_UzTEdulOFs
---
Name: Youth Hostel
Address: 2 Rue du Fort Olisy, Luxembourg
Place ID: ChIJS5A_WTJPlUcRtAr15AXUGNs
---
Name: Novotel Luxembourg Centre
Address: 35 Rue du Laboratoire, Luxembourg
Place ID: ChIJl5q7iM5IlUcROv7uDbapuVo
---
Name: Grand Hotel Cravat
Address: 29 Boulevard Franklin Delano Roosevelt, Luxembourg
Place ID: ChIJi-jQXNNIlUcR77bXMeT_HSw
---
Name: Best Western Plus
Address: 3 Avenue Victor Hugo, Luxembourg
Place ID: ChIJld6G1ClPlUcRbXoEwx9_WUs
---
Name: Meliá Luxembourg
Address: 1 Par

In [ ]:
places['results'][0]

In [14]:
place_id = 'ChIJ3e9sj9VIlUcRUU6bnYZ2tcE'  # Example place ID

details_url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={API_KEY}"

details_response = requests.get(details_url)
place_details = details_response.json()


print(f"Name: {place_details['result']['name']}")
print(f"Address: {place_details['result']['formatted_address']}")
print(f"Phone Number: {place_details['result'].get('formatted_phone_number', 'N/A')}")
print(f"Website: {place_details['result'].get('website', 'N/A')}")


Name: Hôtel Parc Belle-Vue
Address: 5 Av. Marie-Thérèse, 2132 Hollerich Luxembourg
Phone Number: 45 61 41 1
Website: https://www.goereshotels.com/belle-vue-en/


In [17]:
place_details['status']

'OK'

In [7]:
# Connect to DuckDB
# con = duckdb.connect('places.db')
# df = con.execute('SELECT * FROM places').fetchdf()

In [3]:
conn = duckdb.connect('../../data/raw/database.db')
%sql conn --alias duckdb

In [18]:
%%sql
select column_name from  information_schema.columns
where table_name = 'places';

Running query in 'duckdb'

column_name


In [31]:
%%sql
select * from places
limit 10

Running query in 'duckdb'

InvalidInputException: Invalid Input Error: Python Object "places" of type "list" found on line "/tmp/ipykernel_31947/1915770065.py:1" not suitable for replacement scans.
Make sure that "places" is either a pandas.DataFrame, duckdb.DuckDBPyRelation, pyarrow Table, Dataset, RecordBatchReader, Scanner, or NumPy ndarrays with supported format

In [4]:
%%sql
show tables;
# show placedetail;

Running query in 'duckdb'

name
geonames


In [1]:
# con.close()
conn.close()

NameError: name 'conn' is not defined

In [13]:
formatted_location = format_coordinates(49.611671447753906, 6.130000114440918)

In [14]:
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={formatted_location}&radius={radius}&key={API_KEY}"

response = requests.get(url)
places = response.json()

In [20]:
places["results"]

[{'geometry': {'location': {'lat': 49.61162100000001, 'lng': 6.1319346},
   'viewport': {'northeast': {'lat': 49.6549299282108,
     'lng': 6.202720059650757},
    'southwest': {'lat': 49.56028002711416, 'lng': 6.069020017641492}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/geocode-71.png',
  'icon_background_color': '#7B9EB0',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/generic_pinlet',
  'name': 'Luxembourg',
  'photos': [{'height': 554,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/100990507489305992926">Venula Tharusha</a>'],
    'photo_reference': 'AUc7tXUbsRecytMtw8TF6ZhBaEBeJMz37FScVFxcbfWuIZWn6LJ_uxZ1obU_6x4LsnQocfKXDjIiXj0OoMay7sXXVpYQ110phF8LjlIMAFn1PRJ5Z9KxcYDDMYp4ZYqJ168AZz4tMd5RqNbsNdorHhN4gS3lfd8lPMV3m5j-dDx4TOVU2uPr',
    'width': 800}],
  'place_id': 'ChIJVyzznc1IlUcREG0F0dbRAAQ',
  'reference': 'ChIJVyzznc1IlUcREG0F0dbRAAQ',
  'scope': 'GOOGLE',
  'types': ['locality', 'political'

In [17]:

@dataclass
class Place:
    place_id: str
    name: str
    vicinity: str
    latitude: float
    longitude: float

In [18]:
def convert_to_place(result: dict) -> Place:
    """
    Convert a Google Maps API result to a Place object.

    Args:
        result (dict): The result dictionary from the Google Maps API.

    Returns:
        Place: A Place object representing the place.
    """
    place_id = result["place_id"]
    name = result["name"]
    vicinity = result.get("vicinity", "N/A")
    latitude = result["geometry"]["location"]["lat"]
    longitude = result["geometry"]["location"]["lng"]
    return Place(place_id, name, vicinity, latitude, longitude)

In [25]:
def get_all_places(location: str, radius: int):
        """
        Fetch all places within a given radius of a location.

        Args:
            location (str): The location to search around, in the format "latitude,longitude".
            radius (int): The radius in meters to search within.

        Returns:
            List[Place]: A list of Place objects representing the nearby places.
        """
        all_places = []
        url = f"{base_url}/place/nearbysearch/json?location={location}&radius={radius}&key={api_key}"

        while url:
            try:
                response = requests.get(url)
                response.raise_for_status()
            except requests.exceptions.RequestException as e:
                print(f"Error in nearby search request: {e}")
                break

            places = response.json()
            all_places.extend(places["results"])
            next_page_token = places.get("next_page_token")
            if next_page_token:
                time.sleep(2)  # Add a delay before making the next request
                url = f"{base_url}/place/nearbysearch/json?pagetoken={next_page_token}&key={api_key}"
            else:
                url = None

        return [convert_to_place(result) for result in all_places]

In [27]:
places = get_all_places(formatted_location, 50000)

In [29]:
if places:
    df_places = pd.DataFrame([asdict(gn) for gn in places])

In [30]:
df_places

,place_id,name,vicinity,latitude,longitude
0,ChIJVyzznc1IlUcREG0F0dbRAAQ,Luxembourg,Luxembourg,49.611621,6.131935
1,ChIJ3e9sj9VIlUcRUU6bnYZ2tcE,Hôtel Parc Belle-Vue,"5 Avenue Marie-Thérèse, Luxembourg",49.607684,6.122800
2,ChIJiVm9mNZFlUcR1lpH199XBOA,Hôtel Empire,"34 Place de la Gare, Luxembourg",49.599603,6.132795
3,ChIJ6VucqxJPlUcROccW2iDb5Bw,Parc Hotel Alvisse,"120 Route d'Echternach, Luxembourg",49.639720,6.154082
4,ChIJQVrlNdNIlUcRa3v0FrkGQ-g,Hotel Vauban,"10 Place Guillaume II, Luxembourg",49.610940,6.130641
5,ChIJHeKzA9FIlUcR_UzTEdulOFs,Hotel Christophe Colomb,"10 Rue d'Anvers, Luxembourg",49.603400,6.129936
6,ChIJz1niVCtPlUcROFfnez0PXz0,French Café Hotel,"14 Place d'Armes, Luxembourg",49.611567,6.129052
7,ChIJS5A_WTJPlUcRtAr15AXUGNs,Youth Hostel,"2 Rue du Fort Olisy, Luxembourg",49.613236,6.137996
8,ChIJO53gCtBIlUcR_r3IKRd9fMM,ibis Styles Luxembourg Centre Gare,"30 Rue Joseph Junck, Luxembourg",49.600823,6.131229
9,ChIJtZafb69FlUcRBWfwCK9llyQ,Mandarina Hotel Luxembourg Airport,22 Route de Trèves,49.640573,6.217372
